In [16]:
import pandas as pd
import pycountry

In [22]:
df = pd.read_csv("../output/exports.csv")[["country", "country_of_destination", "year","annex_3", "annex_4_a", "annex_4_b", "amount"]]

/tmp/ipykernel_46610/3032554876.py:1: DtypeWarning: Columns (5,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../output/exports.csv")[["country", "country_of_destination", "year","annex_3", "annex_4_a", "annex_4_b", "amount"]]


In [23]:
def func(x):
    try:
        # removing whitespace so country names get recognized properly
        temp = x.strip()
        temp = pycountry.countries.get(alpha_2=temp).name
        return temp
    except:
        return x

df["country_of_destination"] = df["country_of_destination"].apply(func)

In [27]:
df.loc[(df.country_of_destination == "D10")]

,country,country_of_destination,year,annex_3,annex_4_a,annex_4_b,amount
35532,NaN,D10,2014,"H6.1,H12",NaN,Australia,FI
35533,NaN,D10,2014,"H6.1,H12",NaN,Australia,SE
35534,NaN,D10,2014,"H6.1,H12",NaN,Australia,NaN
35536,NaN,D10,2014,"H6.1,H12",NaN,Australia,FI
35537,NaN,D10,2014,"H6.1,H12",NaN,Australia,FI
...,...,...,...,...,...,...,...
40711,NaN,D10,2008,NaN,NaN,Portugal,"ES,FR"
40719,NaN,D10,2008,"H6.1,H12,H3",NaN,United Arab Emirates,"OM,SA,EG,MA,ES,GB,BE,NL,DE"
40720,NaN,D10,2008,"H3,H6.1,H12",NaN,United Arab Emirates,"OM,SA,MA,EG,ES,GB,BE,DE,NL"
40764,NaN,D10,2008,"H3,H6.1,H12",NaN,United Arab Emirates,"OM,SA,EG,MA,ES,GB,BE,NL,DE"
